# Part 2: Let's Help Santa

In [92]:
library("igraph")

### Question 6:

In [95]:
g <- read.graph("./data/uber/uberGraph.txt", format="ncol", directed=FALSE)
info <- sprintf("It has %d nodes and %d edges", vcount(g), ecount(g))
cat(info)

It has 1887 nodes and 311812 edges

In [96]:
# Add attributes

ID_streetName <- rep("", vcount(g))
ID_coordinate <- rep(list(c(1, 2)), vcount(g))
file_uberGraph_info <- file("./data/uber/uberGraph_info.txt", open="r")
line <- readLines(file_uberGraph_info, 1, encoding="latin1")

while(length(line) != 0) {
    line <- strsplit(line, "\t\t")
    ID <- line[[1]][1]
    streetName <- line[[1]][2]
    coordinate <- list(c(as.numeric(line[[1]][3]), as.numeric(line[[1]][4])))

    node_id <- V(g)[V(g)$name == ID]
    ID_streetName[node_id] <- streetName
    ID_coordinate[node_id] <- coordinate
    
    line <- readLines(file_uberGraph_info, 1, encoding="latin1")
} 

close(file_uberGraph_info)
V(g)$displayName <- ID_streetName
V(g)$location <- ID_coordinate

In [97]:
g.components <- clusters(g)
giant.idx <- which.max(g.components$csize)
g <- induced.subgraph(g, which(g.components$membership == giant.idx))

info <- sprintf("It has %d nodes and %d edges", vcount(g), ecount(g))
cat(info)

It has 1880 nodes and 311802 edges

### Question 7:

In [5]:
MST <- mst(g)

### Question 11-14:

In [35]:
# print(V(g)[1]$name)
# print("11")
# print(V(g)$location[[1]][1])
# print(V(g)$location[[1]][2])
# print("22")
# print(V(g)$displayName[1])

graphFile<-file("./data/uber/origin_graph.txt","w")

for (i in 1:1880)
{
    cat(V(g)$name[i], file=graphFile)
    cat("\t\t", file=graphFile)
    cat(V(g)$location[[i]][1],file=graphFile)
    cat("\t\t",file=graphFile)
    cat(V(g)$location[[i]][2],file=graphFile)
    cat("\n",file=graphFile)
    #info <- c(V(g)$name[i],  V(g)$location[[i]][1],  V(g)$location[[i]][2])
}
close(graphFile)

In [152]:
Dg <- read.graph("./data/uber/Delaunay.txt", format="ncol", directed=FALSE)
info <- sprintf("It has %d nodes and %d edges", vcount(g), ecount(g))
cat(info)

It has 1880 nodes and 311802 edges

In [139]:
flowrate <- function(shortest, traveltime) {
    car.speed <- shortest / traveltime
    fr <- 3600 / (0.003/car.speed + 2)
    fr
}

In [142]:
for (e in E(Dg))
{
#     print(e)
    verts = ends(g,e)
    node1 = which(V(g)$name == verts[1])
    node2 = which(V(g)$name == verts[2])

    edge <- get.edge.ids(g, c(node1,node2))
    w<- -1
    if(edge != 0)
    {
        w<-E(g)[edge]$weight
        x1 = V(g)[node1]$location[[1]][1]
        y1 = V(g)[node1]$location[[1]][2]
        x2 = V(g)[node2]$location[[1]][1]
        y2 = V(g)[node2]$location[[1]][2]
        dist = sqrt((x2-x1)*(x2-x1)+(y2-y1)*(y2-y1))*69
        time = w
    }
    else
    {
        w<-shortest_paths(g, from = node1, to = node2, mode = "all",output = "both",)
        dist<-0
        time<-0
        for(pair in w$vpath)
        {
            x1 = V(g)[pair[1]]$location[[1]][1]
            y1 = V(g)[pair[1]]$location[[1]][2]
            x2 = V(g)[pair[2]]$location[[1]][1]
            y2 = V(g)[pair[2]]$location[[1]][2]
            dist = dist + sqrt((x2-x1)*(x2-x1)+(y2-y1)*(y2-y1))*69
        }
        for(edge in w$epath)
        {
            time=time+E(g)[edge]$weight
        }
    }
    E(Dg)[e]$carFlow = flowrate(dist, time)
}

[1] 1793.193


In [157]:
node1 = which(V(g)$displayName == "100 Campus Drive, Stanford")
node2 = which(V(g)$displayName == "700 Meder Street, Santa Cruz")
n1 = which(V(Dg)$name == V(g)[node1]$name)
n2 = which(V(Dg)$name == V(g)[node2]$name)
print(edge_connectivity(Dg, source = n1, target = n2))

[1] 394
[1] 10


In [155]:
w<-shortest_paths(Dg, from = n1, to = n2, mode = "all",output = "both",)
flow<-0
for(edge in w$epath)
{
    flow=flow+E(Dg)[edge]$carFlow
}



[1] 0
